# 멜론차트 TOP100

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

- 1. 데이터 가져오기

In [3]:
url = 'https://www.melon.com/chart/index.htm'
res = requests.get(url)
res.text

''

In [4]:
# Chrome User-Agent (나 로봇아니야!, 구글에 크롬에이전트 window 데이터로 불러옴)
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
res.text[:500]

'<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<head>\r\n\t\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n\t\r\n\r\n\t\r\n\r\n\t\r\n\r\n\t<title>멜론차트>TOP100>멜론</title>\r\n\t<meta name="keywords" content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" />\r\n\t<meta name="description" content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" />\r\n\t<meta name="nave'

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')

- 2. 원하는 데이터 찾기

In [6]:
# 파일이름을 이렇게 쓰려고 타임데이터 가져옴
date_str = soup.select_one('.yyyymmdd').get_text().strip() + \
           soup.select_one('.hhmm').get_text().strip()
date_str.replace('.','').replace(':','')

'202309181000'

In [7]:
trs = soup.select('.lst50')
len(trs)

50

In [8]:
len(soup.select('.lst100'))

50

In [9]:
trs = soup.select('.lst50') +soup.select('.lst100')
len(trs)

100

- 3. 하나를 선택해서 원하는 정보 추출

In [11]:
tr = trs[0]
rank = int(tr.select_one('.rank').get_text().strip())
rank

1

In [14]:
# select_one 안에 공백있으면 에러남으로 공백없애주기(클래스앞에는 항상 .)
title = tr.select_one('.ellipsis.rank01').get_text().strip()
title

'Love Lee'

In [21]:
#.ellipsis.rank02에 뮤지션 이름이 두번 반복되어 결과값도 2번 나옴
artist = tr.select_one('.ellipsis.rank02').get_text().strip()
artist

"""
<div class="ellipsis rank02">										
    <a href="javascript:melon.link.goArtistDetail('712452');" title="AKMU (악뮤) - 페이지 이동">AKMU (악뮤)</a><span class="checkEllipsis" style="display: none;"><a href="javascript:melon.link.goArtistDetail('712452');" title="AKMU (악뮤) - 페이지 이동">AKMU (악뮤)</a></span>
</div>
"""

'AKMU (악뮤)AKMU (악뮤)'

In [22]:
# 자식태그를 이용하여 하나의 데이터만 가져옴
artist = tr.select_one('.ellipsis.rank02 > a').get_text().strip()
artist

'AKMU (악뮤)'

In [23]:
album = tr.select_one('.ellipsis.rank03').get_text().strip()
album

'Love Lee'

In [26]:
# 좋아요는  BeautifulSoup으로 크롤링 불가
like = tr.select_one('.cnt').get_text()
like

'\n총건수\r\n\t\t\t\t\t\t\t\t\t\t\t\t0\r\n\t\t\t\t\t\t\t\t\t\t\t'

![Alt text](image-2.png)

copy selector : #lst50 > td:nth-child(4) > div > a > img

In [31]:
img = tr.select_one('tr > td:nth-child(4) > div > a > img')['src']
img

'https://cdnimg.melon.co.kr/cm2/album/images/113/09/190/11309190_20230818161008_500.jpg/melon/resize/120/quality/80/optimize'

- 4. 한 페이지에 있는 정보 가져오기

In [32]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    title = tr.select_one('.ellipsis.rank01').get_text().strip()
    artist = tr.select_one('.ellipsis.rank02 > a').get_text().strip()
    album = tr.select_one('.ellipsis.rank03').get_text().strip()
    img = tr.select_one('tr > td:nth-child(4) > div > a > img')['src']
    data.append({'순위':rank, '제목':title, '아티스트':artist, '앨범':album, '이미지':img})

- 5. 에러 처리
- 6. 모든 페이지
- 7. 데이터 프레임 만들고 저장하기

In [40]:
df = pd.DataFrame(data)
df.head()

,순위,제목,아티스트,앨범,이미지
0,1,Love Lee,AKMU (악뮤),Love Lee,https://cdnimg.melon.co.kr/cm2/album/images/11...
1,2,"Smoke (Prod. Dynamicduo, Padi)",다이나믹 듀오,스트릿 우먼 파이터2(SWF2) 계급미션,https://cdnimg.melon.co.kr/cm2/album/images/11...
2,3,Seven (feat. Latto) - Clean Ver.,정국,Seven (feat. Latto) - Clean Ver.,https://cdnimg.melon.co.kr/cm2/album/images/11...
3,4,후라이의 꿈,AKMU (악뮤),Love Lee,https://cdnimg.melon.co.kr/cm2/album/images/11...
4,5,Super Shy,NewJeans,NewJeans 2nd EP 'Get Up',https://cdnimg.melon.co.kr/cm2/album/images/11...


In [41]:
df.to_csv(f'data/melon_{date_str}.csv', index=False)
pd.read_csv(f'data/melon_{date_str}.csv').tail

<bound method NDFrame.tail of      순위                                제목       아티스트  \
0     1                          Love Lee  AKMU (악뮤)   
1     2    Smoke (Prod. Dynamicduo, Padi)    다이나믹 듀오   
2     3  Seven (feat. Latto) - Clean Ver.         정국   
3     4                            후라이의 꿈  AKMU (악뮤)   
4     5                         Super Shy   NewJeans   
..  ...                               ...        ...   
95   96                첫 키스에 내 심장은 120BPM         경서   
96   97                      주저하는 연인들을 위해        잔나비   
97   98                         사랑..그게 뭔데         지아   
98   99                      Yogurt Shake  NCT DREAM   
99  100                              Blue          V   

                                  앨범  \
0                           Love Lee   
1             스트릿 우먼 파이터2(SWF2) 계급미션   
2   Seven (feat. Latto) - Clean Ver.   
3                           Love Lee   
4           NewJeans 2nd EP 'Get Up'   
..                               ...   
95               